# **Códificación Shannon Fano**

*La codificación Shannon-Fano es un algoritmo de compresión de datos sin pérdida desarrollado por Robert Fano a partir de una idea de Claude Shannon.*

**Se trata de una codificación de entropía que produce un código de prefijo muy similar a un código de Huffman , aunque no siempre óptimo, a diferencia de este último.**

Un árbol Shannon-Fano se construye de acuerdo a una especificación diseñada para definir una tabla de códigos efectiva. El algoritmo actual es simple:

1. Para una lista de símbolos dada, crear su correspondiente lista de probabilidades o de frecuencias de aparición de manera que se conozca la frecuencia relativa de ocurrencia de cada símbolo.

2. Ordenar las listas de símbolos de acuerdo a la frecuencia, con los símbolos de ocurrencia más frecuente a la izquierda y los menos comunes a la derecha.

3. Dividir la lista en dos partes, haciendo la frecuencia total de la mitad izquierda lo más próxima posible a la de la mitad derecha.

4. Asignar a la mitad izquierda el dígito binario “0”, y a la mitad derecha el dígito “1”. Esto significa que los códigos para los símbolos en la primera mitad empezarán con “0”, y que los códigos de la segunda mitad empezarán por “1”.
5. Aplicar recursivamente los pasos 3 y 4 a cada una de las dos mitades, subdividiéndolas en grupos y añadiendo bits a los códigos hasta que cada símbolo se corresponde con una hoja del árbol.

### 1. Fuente de información

In [15]:
# 1. Fuente de información (leyendo un archivo de texto)

def leer_archivo(nombre_archivo):
    try:
        with open(nombre_archivo, 'r') as archivo:
            contenido = archivo.read()
            return contenido
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return ""

nombre_archivo_fuente = "fuente.txt"

texto_original = leer_archivo(nombre_archivo_fuente)

print("Mensaje:", texto_original)

Mensaje: Hola, esto es un mensaje de prueba para la comunicacion.


### 2. Transmisor

In [16]:
# Paso 1: Convertir el mensaje de texto a una cadena binaria
def texto_a_binario(mensaje):
    binario = ''.join(format(ord(char), '08b') for char in mensaje)
    return binario

mensaje_binario = texto_a_binario(texto_original)

In [17]:
# Paso 2: Dividir la cadena binaria en bytes (8 bits)
def dividir_binario_en_bytes(binario):
    return [binario[i:i+8] for i in range(0, len(binario), 8)]

bytes_binarios = dividir_binario_en_bytes(mensaje_binario)

In [18]:
# Paso 3: Calcular las frecuencias de cada byte
from collections import defaultdict

def calcular_frecuencias(bytes):
    frecuencias = defaultdict(int)
    for byte in bytes:
        frecuencias[byte] += 1
    return frecuencias

frecuencias = calcular_frecuencias(bytes_binarios)

In [19]:
# Paso 4: Construir la tabla de codificación Shannon-Fano
def construir_tabla_shannon_fano(frecuencias):
    # Ordenar las frecuencias en orden descendente
    frecuencias_ordenadas = sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)
    
    # Crear la tabla de codificación Shannon-Fano
    tabla_codificacion = {}
    def asignar_codigos(frecuencias_ordenadas, codigo=''):
        if len(frecuencias_ordenadas) == 1:
            tabla_codificacion[frecuencias_ordenadas[0][0]] = codigo
        else:
            split_point = len(frecuencias_ordenadas) // 2
            for byte, frecuencia in frecuencias_ordenadas[:split_point]:
                tabla_codificacion[byte] = codigo + '0'
            for byte, frecuencia in frecuencias_ordenadas[split_point:]:
                tabla_codificacion[byte] = codigo + '1'
            asignar_codigos(frecuencias_ordenadas[:split_point], codigo + '0')
            asignar_codigos(frecuencias_ordenadas[split_point:], codigo + '1')
    
    asignar_codigos(frecuencias_ordenadas)
    return tabla_codificacion

tabla_codificacion_sf = construir_tabla_shannon_fano(frecuencias)

In [20]:
# Paso 5: Codificar el mensaje utilizando la tabla de codificación Shannon-Fano
def codificar_con_shannon_fano(bytes_binarios, tabla_codificacion):
    texto_codificado = ''.join(tabla_codificacion[byte] for byte in bytes_binarios)
    return texto_codificado

mensaje_codificado = codificar_con_shannon_fano(bytes_binarios, tabla_codificacion_sf)

### 3. Canal

In [21]:
# 3. Canal (agregar ruido a la cadena binaria)

### 4. Receptor

In [22]:
# Paso 6: Decodificar el texto codificado utilizando la tabla de codificación Shannon-Fano
def decodificar_con_shannon_fano(texto_codificado, tabla_codificacion):
    texto_decodificado = ''
    codigo_actual = ''
    for bit in texto_codificado:
        codigo_actual += bit
        if codigo_actual in tabla_codificacion.values():
            byte = [key for key, value in tabla_codificacion.items() if value == codigo_actual][0]
            texto_decodificado += byte
            codigo_actual = ''
    return texto_decodificado

mensaje_decodificado = decodificar_con_shannon_fano(mensaje_codificado, tabla_codificacion_sf)

### 5. Destino de información

In [24]:
# 5. Destino de Información (Destinatario)
print("Mensaje Original:", texto_original)
print("Mensaje Codificado:", mensaje_codificado)
print("Mensaje Decodificado:", mensaje_decodificado)

Mensaje Original: Hola, esto es un mensaje de prueba para la comunicacion.
Mensaje Codificado: 101100011001110000110111000000100100110000110000000100100000001010011100000111100100011101000001110100100000111000100000100010010101001011110000100001000000110010001000001110000100000110001100111101010011110100110000101101010001100011111111
Mensaje Decodificado: 0100100001101111011011000110000100101100001000000110010101110011011101000110111100100000011001010111001100100000011101010110111000100000011011010110010101101110011100110110000101101010011001010010000001100100011001010010000001110000011100100111010101100101011000100110000100100000011100000110000101110010011000010010000001101100011000010010000001100011011011110110110101110101011011100110100101100011011000010110001101101001011011110110111000101110


In [26]:
print("Tabla de Codificación Shannon-Fano:")
for byte, codigo in tabla_codificacion_sf.items():
    print(f"{byte}: {codigo}")


Tabla de Codificación Shannon-Fano:
00100000: 0000
01100001: 0001
01100101: 0010
01101111: 00110
01101110: 00111
01110011: 0100
01110101: 0101
01100011: 0110
01101100: 01110
01101101: 01111
01110000: 1000
01110010: 1001
01101001: 1010
01001000: 10110
00101100: 10111
01110100: 1100
01101010: 1101
01100100: 1110
01100010: 11110
00101110: 11111
